## Data Preprocessing

### Changing File Names
The file names of the images in the dataset are changed to a more readable format. The original file names are in the format `01.jpg`, `02.jpg`, etc. The new file names are in the format `01_pants.jpg`, `02_pants.jpg`, etc.

In [ ]:
%%bash
# Rename shorts
cd clothes/train/shorts
for i in {1..40}; do mv "$(printf "%02d" $i).jpg" "$(printf "%02d" $i)_shorts.jpg"; done

# Rename pants
cd ../pants
for i in {1..40}; do mv "$(printf "%02d" $i).jpg" "$(printf "%02d" $i)_pants.jpg"; done

# Rename shirts
cd ../shirts
for i in {1..40}; do mv "$(printf "%02d" $i).jpg" "$(printf "%02d" $i)_shirts.jpg"; done

# Rename t-shirts
cd ../t-shirts
for i in {1..40}; do mv "$(printf "%02d" $i).jpg" "$(printf "%02d" $i)_t-shirts.jpg"; done
done

### Kaggle Dataset
<table>
  <tr>
    <td style="vertical-align:top; width: 50%;">
        <img src="./images/15.jpg" width="100%"><br>
    </td>
    <td style="vertical-align:top; width: 50%;">
        <img src="./images/07.jpg" width="100%"><br>
    </td>
  </tr>
</table>

We trained our agent using Kaggle’s clothing dataset, categorized into pants, shorts, shirts, and t-shirts.

### Metadata

![metadata](images/metadata.png)

Metadata includes the following:
- File name
- Categories: The type of clothing (e.g., pants, shorts, shirts, t-shirts)
- Color: The color of the clothing item
- Style: The occasion for the clothing item (e.g., casual, formal, sporty)
- Gender: The gender for the clothes (e.g., male, female, unisex)
- Sleeve length: The length of the sleeves (e.g., short, long, sleeveless)




In [4]:
import os
from pillow_heif import register_heif_opener
from PIL import Image

# Register HEIC opener
register_heif_opener()

# Specify the file path of the HEIC image
heic_file = "images/pants_01.heic"  # Replace with the actual file path
jpg_file = heic_file.rsplit('.', 1)[0] + '.jpg'

# Open and convert the image
try:
    img = Image.open(heic_file)
    img.save(jpg_file, "JPEG")
    print(f"Converted: {os.path.basename(heic_file)} -> {os.path.basename(jpg_file)}")
except Exception as e:
    print(f"Failed to convert {os.path.basename(heic_file)}: {e}")

Converted: pants_01.heic -> pants_01.jpg


### Image Segmentation

<table>
  <tr>
    <td style="vertical-align:top; width: 50%;">
        <img src="./images/pants_01.jpg" width="100%"><br>
    </td>
    <td style="vertical-align:top; width: 50%;">
        <img src="./images/01_pants.jpg" width="100%"><br>
    </td>
  </tr>
</table>

We used HuggingFace's `segment-anything` library to segment the clothing items in the images. The segmentation masks are saved in a separate folder for each image. The masks are used to create a binary mask for each clothing item, which is then used to train the agent.

In [ ]:
from PIL import Image
import os

_GPU_INDEX = 0

example_input_path = "./clothes/test/pants/01_pants.jpg"
example_dir = ".clothes/test/pants"

os.makedirs(example_dir, exist_ok=True)
input_raw = Image.open(example_input_path)

# show the input image
input_raw_copy = input_raw.copy()
input_raw_copy.thumbnail((256, 256))
input_raw_copy

# initialize the Segment Anything model
predictor = sam_init(_GPU_INDEX)

# preprocess the input image
input_256 = preprocess(predictor, input_raw)
input_256

import os
import torch

# GPU Index (Set to 0 if using CUDA, otherwise it'll default to CPU)
_GPU_INDEX = 0

# Root directory for dataset
root_dir = "./clothes/test"
categories = ["pants", "shirts", "shorts", "t-shirts"]

# Initialize the Segment Anything Model (SAM)
predictor = sam_init(_GPU_INDEX)

def preprocess(predictor, raw_im, lower_contrast=False):
    """Preprocess image using SAM."""
    raw_im.thumbnail([512, 512], Image.Resampling.LANCZOS)
    image_sam = sam_out_nosave(predictor, raw_im.convert("RGB"), pred_bbox(raw_im))
    input_256 = image_preprocess_nosave(image_sam, lower_contrast=lower_contrast, rescale=True)
    torch.cuda.empty_cache()
    return input_256

# Iterate through each category (pants, shirt, shorts, t-shirt)
for category in categories:
    input_folder = os.path.join(root_dir, category)
    output_folder = os.path.join(root_dir, f"{category}_segmented")

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process all images in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            # Open and process the image
            input_raw = Image.open(image_path)
            processed_image = preprocess(predictor, input_raw)

            # Save processed image
            processed_image.save(output_path)
            print(f"Saved: {output_path}")

print("All images processed and saved!")